# Electric Vehicle Population Data for Washington State

### Current registered EVs in the State of Washington according to Washington State Department of Licensing (DOL)


## <ins>Introduction: Cleaning and exploring the dataset</ins>

In [2]:
import pandas as pd
import seaborn as sns
import numpy as np

In [8]:
df = pd.read_csv('./csv/Electric_Vehicle_Population_Data.csv')

print(len(df.columns))
print(df.info())
df.head(1)

17
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 247344 entries, 0 to 247343
Data columns (total 17 columns):
 #   Column                                             Non-Null Count   Dtype  
---  ------                                             --------------   -----  
 0   VIN (1-10)                                         247344 non-null  object 
 1   County                                             247340 non-null  object 
 2   City                                               247340 non-null  object 
 3   State                                              247344 non-null  object 
 4   Postal Code                                        247340 non-null  float64
 5   Model Year                                         247344 non-null  int64  
 6   Make                                               247344 non-null  object 
 7   Model                                              247344 non-null  object 
 8   Electric Vehicle Type                              247344 non-null  obj

,VIN (1-10),County,City,State,Postal Code,Model Year,Make,Model,Electric Vehicle Type,Clean Alternative Fuel Vehicle (CAFV) Eligibility,Electric Range,Base MSRP,Legislative District,DOL Vehicle ID,Vehicle Location,Electric Utility,2020 Census Tract
0,1N4BZ0CP5G,King,Seattle,WA,98125.0,2016,NISSAN,LEAF,Battery Electric Vehicle (BEV),Clean Alternative Fuel Vehicle Eligible,84.0,0.0,46.0,349455557,POINT (-122.30253 47.72656),CITY OF SEATTLE - (WA)|CITY OF TACOMA - (WA),5.303300e+10


### Checking various columns to see how many unique values there are
some of these columns Ill probably be able to convert to categories instead of strings/objects for better performance
Also based on the info method call above, and looking at column types, I might convert other column types where it makes sense

As an example, some of the float columns would make more sense if they are ints, i.e. postal code

In [12]:
df['Clean Alternative Fuel Vehicle (CAFV) Eligibility'].unique()


array(['Battery Electric Vehicle (BEV)',
       'Plug-in Hybrid Electric Vehicle (PHEV)'], dtype=object)

In [25]:
df['Electric Vehicle Type'].unique()

array(['Battery Electric Vehicle (BEV)',
       'Plug-in Hybrid Electric Vehicle (PHEV)'], dtype=object)

In [16]:
df['Electric Utility'].unique()

75

In [24]:
df['State'].unique()

array(['WA', 'BC', 'CA', 'NY', 'OR', 'WY', 'CO', 'MD', 'FL', 'NC', 'MN',
       'VA', 'AZ', 'IL', 'NJ', 'GU', 'TX', 'NE', 'MO', 'IN', 'MA', 'AP',
       'MI', 'GA', 'ID', 'UT', 'DC', 'SC', 'CT', 'AR', 'RI', 'NV', 'PA',
       'KS', 'LA', 'OK', 'AL', 'HI', 'KY', 'NH', 'OH', 'NM', 'TN', 'ME',
       'WI', 'DE', 'NS', 'MS', 'AK'], dtype=object)

In [17]:
df['Clean Alternative Fuel Vehicle (CAFV) Eligibility'].unique()

array(['Clean Alternative Fuel Vehicle Eligible',
       'Not eligible due to low battery range',
       'Eligibility unknown as battery range has not been researched'],
      dtype=object)

Converting columns

In [33]:
df['State'] = df['State'].astype('category')
df['Electric Vehicle Type'] = df['Electric Vehicle Type'].astype('category')
df['Clean Alternative Fuel Vehicle (CAFV) Eligibility'] = df['Clean Alternative Fuel Vehicle (CAFV) Eligibility'].astype('category')
df['Postal Code'] = df['Postal Code'].astype('Int64')
df['Electric Range'] = df['Electric Range'].astype('Int64')
df['Legislative District'] = df['Legislative District'].astype('Int64')

df.info()


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 247344 entries, 0 to 247343
Data columns (total 17 columns):
 #   Column                                             Non-Null Count   Dtype   
---  ------                                             --------------   -----   
 0   VIN (1-10)                                         247344 non-null  object  
 1   County                                             247340 non-null  object  
 2   City                                               247340 non-null  object  
 3   State                                              247344 non-null  category
 4   Postal Code                                        247340 non-null  Int64   
 5   Model Year                                         247344 non-null  int64   
 6   Make                                               247344 non-null  object  
 7   Model                                              247344 non-null  object  
 8   Electric Vehicle Type                              247344 non-nu